In [ ]:
from keras.applications.vgg16 import VGG16
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import zipfile
import os
import keras
import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import matplotlib as mpl

In [ ]:
# Para extraer las imágenes Glaucoma
drive.mount('/content/drive')

# Ruta a la carpeta donde están tus archivos zip
base_zip_path = ('/content/drive/My Drive/Redes_Neuronales/')
# Directorios para extraer los contenidos
data_dir = '/content/train'
test_dir = '/content/validation'

# Descomprimir data.zip
with zipfile.ZipFile(base_zip_path + 'Train_Glau.zip', 'r') as zip_ref:
    zip_ref.extractall(data_dir)
# Descomprimir test.zip
with zipfile.ZipFile(base_zip_path + 'Validation_Glau.zip', 'r') as zip_ref:
    zip_ref.extractall(test_dir)

In [ ]:
def nuevo_modelo():
  model = VGG16(include_top = False, input_shape = (224,224,3))
  for layer in model.layers:
    # ahora no aprenden:
    layer.trainable = False
  # añadimos nuevas:
  capa_flat = Flatten()(model.layers[-1].output)
  densa1 = Dense(128, activation = "relu", kernel_initializer = "he_uniform")(capa_flat)
  salida = Dense(1, activation = "sigmoid")(densa1)
  # estructura:
  model = Model(inputs = model.inputs, outputs = salida)
  # optimizador:
  opt = Adam(learning_rate=0.0001, beta_1=0.9)
  model.compile(opt, loss = "binary_crossentropy", metrics = ["accuracy"])
  return model

In [ ]:
# Curvas de diagnóstico:
def graficas_f (history):
 # loss
 pyplot.subplot(211)
 pyplot.title('Cross Entropy Loss')
 pyplot.plot(history.history['loss'], color='blue', label='train')
 pyplot.plot(history.history['val_loss'], color='orange', label='test')
 # accuracy
 pyplot.subplot(212)
 pyplot.title('Classification Accuracy')
 pyplot.plot(history.history['accuracy'], color='blue', label='train')
 pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
 # guardamos
 filename = sys.argv[0].split('/')[-1]
 pyplot.savefig(filename + '_plot.png')
 pyplot.close()

In [ ]:
def entrenamiento_modelo():
  # creamos el modelo con la función:
  model = nuevo_modelo()

  datagen = ImageDataGenerator(featurewise_center = True)
  datagen.mean = [123.68, 116.779, 103.939]

  # preparamos el iterador:
  traindata = datagen.flow_from_directory(directory=data_dir, class_mode='binary', batch_size=64, target_size=(224, 224))

  testdata = datagen.flow_from_directory(directory=test_dir, class_mode='binary', batch_size=64, target_size=(224, 224))

  # model fit
  history = model.fit(traindata,
                      steps_per_epoch=len(traindata),
                      validation_data=testdata,
                      validation_steps=len(testdata),
                      epochs=10,
                      verbose=1)
   # lo evaluamos
  _, acc = model.evaluate(testdata, steps=len(testdata), verbose=0)
  print('> %.3f' % (acc * 100.0))
  #curvas
  graficas_f (history)
  # guardamos el modelo
  model.save('modelo_final.keras')

In [ ]:
entrenamiento_modelo()

In [ ]:
from keras.models import load_model
model = load_model('modelo_final.keras')

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_structure.png', show_shapes=True)